# Batch Processing CCTV-POS Mapping CSV Files
This script is designed to read the various files in the folder, clean them, and format the data tables to match the following form:

| NVR | IPv4 | POSnum | Channel | Camera |
|----------|----------|----------|----------|----------|
|   NVR012304  |   192.168.1.101  |   4001  |   02  |   Register Bay 3  |
|   NVR999912  |   10.0.0.25  |   4023  |   33  |   POS 1  |
|   NVR010502  |   172.16.254.1  |   7005  |   64  |   Cashwrap  |
|   NVR020411  |   203.0.113.0  |   7120  |   15  |   Registers  |

### Step 1
Get the file path.


In [1]:
folder_path = input("Enter the folder path: ")

Enter the folder path:  \\A0319P1116\file_repo\inbound\APREGUPDATE


### Step 2
Read all the content in the folder.

In [3]:
import os
import pandas as pd

# Get all files in the folder
files = os.listdir(folder_path)

for file in files:
    # Check if the file is a CSV file
    if file.endswith(".csv"):
        # Read the CSV file as a pandas table
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)

### Step 3
Delete all rows that were submitted by specific users during testing.

In [7]:
# Get user input for user names to delete. Get them one at a time until they're all entered.
user_names = []
while True:
    user_name = input("Enter a user name to delete (or type 'done' to finish): ")
    if user_name == 'done':
        break
    user_names.append(user_name)

# Create a new empty DataFrame
new_df = pd.DataFrame()

# Iterate over the rows in the original DataFrame
for index, row in df.iterrows():
    # Check if the user name is in the list of user names to delete
    if row['USER'] not in user_names:
        # If it's not, add the row to the new DataFrame
        new_df = new_df.append(row, ignore_index=True)

# Replace the original DataFrame with the new one
df = new_df

AttributeError: 'DataFrame' object has no attribute 'append'

### Step 4
Delete all rows that contain a value of "No camera".

In [ ]:
# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Find any rows that contain "No camera" in any column
    if "No camera" in row.values:
        # Delete the row
        df.drop(index, inplace=True)

### Step 5
Reorganize the columns in each row.

In [ ]:
# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Swap the values in column 1 and column 2
    df.at[index, 1], df.at[index, 2] = df.at[index, 2], df.at[index, 1]
    

### Step 6
Look up the IP address for each NVR. The reference file must be a CSV with column 1 as NVR name and column 2 as IP address. Insert the IP address into each row.

In [ ]:
# ask the user for the file path to the NVR IP CSV file
nvr_ip_file = input("Enter the file path to the CSV file containing the NVRs and IP addresses: ")

# Read the CSV file as a pandas table
nvr_ip_df = pd.read_csv(nvr_ip_file)

# Insert a new column with empty values at index 2
df.insert(2, "IPv4", "")

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Get the NVR name from the current row
    nvr_name = row[0]
    # Find the row in the NVR IP DataFrame that has the same NVR name
    nvr_ip_row = nvr_ip_df.loc[nvr_ip_df.iloc[:, 0] == nvr_name]
    # If the row exists, get the IP address from the row
    if not nvr_ip_row.empty:
        ip_address = nvr_ip_row.iloc[0, 1]
        # Insert the IP address between columns 1 and 2 so that column 2 becomes column 3 in the current row of the DataFrame
        df.at[index, 2] = ip_address
